In [4]:
from keras import layers
from keras import models

In [5]:
img_height = 224
img_width = 224
img_channels = 3
cardinality = 32

In [12]:
def residual_network(x):
  def add_common_layers(y):
    y = layers.BatchNormalization()(y)
    y = layers.LeakyReLU()(y)
    return y
  def grouped_convolution(y, nb_channels, _strides):
    if cardinality == 1:
      return layers.Conv2D(nb_channels, kernel_size=(3, 3), strides=_strides, padding='same')(y)
    assert not nb_channels % cardinality
    _d = nb_channels // cardinality
    groups = []
    for j in range(cardinality):
      group = layers.Lambda(lambda z: z[:, :, :, j * _d:j * _d + _d])(y)
      groups.append(layers.Conv2D(_d, kernel_size=(3, 3), strides=_strides, padding='same')(group))
    y = layers.concatenate(groups)
    return y
  def residual_block(y,nb_channels_in, nb_channels_out, _strides=(1, 1), _project_shortcut=False):
    shortcut = y
    y = layers.Conv2D(nb_channels_in, kernel_size=(1, 1), strides=(1, 1), padding='same')(y)
    y = add_common_layers(y)
    y = grouped_convolution(y, nb_channels_in, _strides=_strides)
    y = add_common_layers(y)
    y = layers.Conv2D(nb_channels_out, kernel_size=(1, 1), strides=(1, 1), padding='same')(y)
    y = layers.BatchNormalization()(y)
    if _project_shortcut or _strides != (1, 1):
      shortcut = layers.Conv2D(nb_channels_out, kernel_size=(1, 1), strides=_strides, padding='same')(shortcut)
      shortcut = layers.BatchNormalization()(shortcut)
    y = layers.add([shortcut, y])
    y = layers.LeakyReLU()(y)
    return y
# conv1
  x = layers.Conv2D(64, kernel_size=(7, 7), strides=(2, 2), padding='same')(x)
  x = add_common_layers(x)
# conv2
  x = layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

  for i in range(3):
    project_shortcut = True if i == 0 else False
    x = residual_block(x, 128, 256, _project_shortcut=project_shortcut)
# conv3
  for i in range(4):
    strides = (2, 2) if i == 0 else (1, 1)
    x = residual_block(x, 256, 512, _strides=strides)
# conv4
  for i in range(6):
    strides = (2, 2) if i == 0 else (1, 1)
    x = residual_block(x, 512, 1024, _strides=strides)
# conv5
  for i in range(3):
    strides = (2, 2) if i == 0 else (1, 1)
    x = residual_block(x, 1024, 2048, _strides=strides)
  x = layers.GlobalAveragePooling2D()(x)
  x = layers.Dense(1)(x)
  return x
  image_tensor = layers.Input(shape=(img_height, img_width, img_channels))
  network_output = residual_network(image_tensor)
  model = models.Model(inputs=[image_tensor], outputs=[network_output])
  print(model.summary())